Первый вариант - алгоритм градиентного бустинга CatBoost, автоматический подбор гиперпараметров и регуляризация с использованием Optuna и кросс-валидацией, с масштабированием RobustScaler, без категориального признака 'Sector'.

In [1]:
import polars as pl
import numpy as np
import pandas as pd
import pyarrow

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

In [3]:
# Загружаем датасет с данными
data = pl.read_csv('financials.csv')
data.head()

Symbol,Name,Sector,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book,SEC Filings,Earnings,Sales,Book
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64
"""MMM""","""3M Company""","""Industrials""",222.89,24.31,2.3328617,7.92,259.77,175.49,1.3872e11,9.0480e9,4.3902707,11.34,"""http://www.sec.gov/cgi-bin/bro…",9.168655,50.769079,19.655203
"""AOS""","""A.O. Smith Corp""","""Industrials""",60.24,27.76,1.1479592,1.7,68.39,48.925,1.0783e10,6.01e8,3.5754826,6.35,"""http://www.sec.gov/cgi-bin/bro…",2.170029,16.848075,9.486614
"""ABT""","""Abbott Laboratories""","""Health Care""",56.27,22.51,1.9089824,0.26,64.6,42.28,1.0212e11,5.7440e9,3.7404804,3.19,"""http://www.sec.gov/cgi-bin/bro…",2.499778,15.043522,17.639498
"""ABBV""","""AbbVie Inc.""","""Health Care""",108.48,19.41,2.4995599,3.29,125.86,60.05,1.8139e11,1.0310e10,6.291571,26.14,"""http://www.sec.gov/cgi-bin/bro…",5.588872,17.242116,4.149962
"""ACN""","""Accenture plc""","""Information Technology""",150.51,25.47,1.7144699,5.44,162.6,114.82,9.8766e10,5.6432e9,2.604117,10.62,"""http://www.sec.gov/cgi-bin/bro…",5.909305,57.796942,14.172316


In [4]:
# Выбираем нужные столбцы 
df_polars = data.select(['Sector', 'Dividend Yield', 'Earnings/Share', '52 Week Low', '52 Week High', 'Market Cap', 'EBITDA', 'Earnings', 'Sales', 'Book'])

In [5]:
# Целевая переменная
y = df_polars['Market Cap']

# Признаки (все остальные столбцы, кроме целевой)
X = df_polars.select(['Dividend Yield', 'Earnings/Share', '52 Week Low', '52 Week High', 'EBITDA', 'Earnings', 'Sales', 'Book'])

In [6]:
# Конвертируем Polars DataFrame в Pandas DataFrame
y = y.to_pandas()
X = X.to_pandas()

In [7]:
# Разделяем на train/test (последние 20% для теста)
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]


In [8]:
# Создадим объект класса RobustScaler
scaler = RobustScaler()
scaler

,with_centering,True
,with_scaling,True
,quantile_range,"(25.0, ...)"
,copy,True
,unit_variance,False


In [9]:
# Масштабируем признаки обучающей выборки
X_train_scaled = scaler.fit_transform(X_train)

In [10]:
# Преобразуем тестовые данные с использованием среднего и СКО, рассчитанных на обучающей выборке, так тестовые данные не повляют на обучение модели, и мы избежим утечки данных
X_test_scaled = scaler.transform(X_test)

In [11]:
import optuna

In [12]:
# Импортируем необходимые библиотеки и произведем базовые настройки
from catboost import CatBoostRegressor
from catboost import Pool, cv, sum_models

In [13]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error, r2_score  # Метрики ошибок

In [14]:
# Функция для objective (целевой метрики) в Optuna
def objective(trial):
    # Подбор гиперпараметров через trial
    params = {
        'iterations': trial.suggest_int('iterations', 500, 1500, step=100),             # Количество деревьев
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),     # Скорость обучения
        'depth': trial.suggest_int('depth', 4, 8),                                      # Глубина деревьев
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-2, 10.0, log=True),        # L2 регуляризация
        'random_seed': 42,                                                              # Фиксированное значение random
        'verbose': 0                                                                   # Частота вывода
        # 'cat_features': cat_features                                                # Указываем категориальные признаки прямо в модели
    }
    
    # Создаём модель с текущими параметрами
    regressor = CatBoostRegressor(**params)
    
    # Кросс‑валидация (5 фолдов)
    cv_strategy = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(
        regressor, X_train_scaled, y_train,
        cv=cv_strategy,
        scoring='r2'
    ) 

    # Возвращаем среднее значение метрики (Optuna максимизирует objective)
    return np.mean(scores)

In [15]:
# Создание исследования Optuna
study = optuna.create_study(
    direction='maximize',  # Максимизируем R2
    sampler=optuna.samplers.TPESampler(seed=42)
)

[I 2025-12-13 15:23:05,714] A new study created in memory with name: no-name-2d983fe7-4262-4e4e-84d9-e878a7326cab


In [16]:
# Запуск оптимизации (100 итераций)
study.optimize(objective, n_trials=100, show_progress_bar=True)

  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-12-13 15:23:26,224] Trial 0 finished with value: 0.5194055427271347 and parameters: {'iterations': 900, 'learning_rate': 0.2536999076681772, 'depth': 7, 'l2_leaf_reg': 0.6251373574521749}. Best is trial 0 with value: 0.5194055427271347.
[I 2025-12-13 15:23:29,657] Trial 1 finished with value: 0.49676029778121505 and parameters: {'iterations': 600, 'learning_rate': 0.01699897838270077, 'depth': 4, 'l2_leaf_reg': 3.9676050770529883}. Best is trial 0 with value: 0.5194055427271347.
[I 2025-12-13 15:23:35,708] Trial 2 finished with value: 0.4659856544618548 and parameters: {'iterations': 1100, 'learning_rate': 0.11114989443094977, 'depth': 4, 'l2_leaf_reg': 8.123245085588687}. Best is trial 0 with value: 0.5194055427271347.
[I 2025-12-13 15:23:43,454] Trial 3 finished with value: 0.5483088879343374 and parameters: {'iterations': 1400, 'learning_rate': 0.020589728197687916, 'depth': 4, 'l2_leaf_reg': 0.03549878832196503}. Best is trial 3 with value: 0.5483088879343374.
[I 2025-12-13

In [17]:
# Вывод лучших результатов
print(f"Лучшие параметры: {study.best_params}")
print(f"Лучший результат (R2): {study.best_value:.4f}")

Лучшие параметры: {'iterations': 900, 'learning_rate': 0.0454003744102823, 'depth': 6, 'l2_leaf_reg': 0.42244692292544583}
Лучший результат (R2): 0.5837


In [18]:
# Обучение финальной модели на лучших параметрах
best_model = CatBoostRegressor(
    **study.best_params,
    random_seed=42,
    verbose=0
    # cat_features=cat_features
)
best_model.fit(X_train_scaled, y_train)

In [19]:
# Предсказание
y_pred = best_model.predict(X_test_scaled)

In [23]:
# Вычислим MSE, RMSE и MAE на тестовой выборке
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)  
mape = mean_absolute_percentage_error(y_test, y_pred) 
 
# Рассчитаем Коэффициент детерминации на обучающей и тестовой выборках
r_cat_train = best_model.score(X_train_scaled, y_train)
r_cat_test = best_model.score(X_test_scaled, y_test)

print(f"MSE (Mean Squared Error):  {mse:.2f}")
print(f"RMSE (Root Mean Squared Error): {rmse:.2f}")
print(f"MAE (Mean Absolute Error): {mae:.2f}")
print(f"MAPE (Mean Absolute Percentage Error): {mape:.2f}")

print(f"Коэффициент детерминации на train r^2: {r_cat_train}")
print(f"Коэффициент детерминации на test r^2: {r_cat_test}")

MSE (Mean Squared Error):  1052316856371744997376.00
RMSE (Root Mean Squared Error): 32439433662.93
MAE (Mean Absolute Error): 15367945272.19
MAPE (Mean Absolute Percentage Error): 0.61
Коэффициент детерминации на train r^2: 0.9998987647804097
Коэффициент детерминации на test r^2: 0.6742431443464872
